This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [2]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'mykey'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

In [1]:
# First, import the relevant modules
import requests


In [21]:
# Now, call the Quandl API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X/data.json'
params = dict(api_key = API_KEY, start_date = '2020-04-28', end_date = '2020-04-28')

In [24]:
r = requests.get(url, params = params)
json = r.json()
print(json)

{'dataset_data': {'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2020-04-28', 'end_date': '2020-04-28', 'frequency': 'daily', 'data': [['2020-04-28', 92.0, 92.2, 89.15, 89.65, None, 163949.0, 14804941.85, None, None, None]], 'collapse': None, 'order': None}}


In [30]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure
print(json['dataset_data'])

{'limit': None, 'transform': None, 'column_index': None, 'column_names': ['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'], 'start_date': '2020-04-28', 'end_date': '2020-04-28', 'frequency': 'daily', 'data': [['2020-04-28', 92.0, 92.2, 89.15, 89.65, None, 163949.0, 14804941.85, None, None, None]], 'collapse': None, 'order': None}


These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [37]:
#1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
params = dict(api_key = API_KEY, start_date = '2017-01-01', end_date = '2017-12-31')
r = requests.get(url, params = params)
print(r)

<Response [200]>


In [94]:
#1.Convert the returned JSON object into a Python dictionary.
json = r.json()
type(json)

dict

In [95]:
#2.Convert the returned JSON object into a Python dictionary.

from collections import defaultdict

columns = json['dataset_data']['column_names']
lines = json['dataset_data']['data']

#construct the dictionary using 
data = defaultdict(list)

for line in lines:
    for i in range(len(line)):
        data[columns[i]].append(line[i])

print(type(data))
print(data.keys())

<class 'collections.defaultdict'>
dict_keys(['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover'])


In [127]:
#Check if None exists in any of the columns
for column in columns:
    check = None in data[column]
    print('If', column, 'has None: ', check)

If Date has None:  False
If Open has None:  True
If High has None:  False
If Low has None:  False
If Close has None:  False
If Change has None:  True
If Traded Volume has None:  False
If Turnover has None:  False
If Last Price of the Day has None:  True
If Daily Traded Units has None:  True
If Daily Turnover has None:  True


In [128]:
#Remove None from the Open column
open_price = [x for x in data['Open'] if x != None]
print(len(open_price))

252


In [129]:
#3. Calculate what the highest and lowest opening prices were for the stock in this period.
highest_opening = max(open_price)
lowest_opening = min(open_price)

print(highest_opening)
print(lowest_opening)

53.11
34.0


In [137]:
#4. What was the largest change in any one day (based on High and Low price)?
change_same_day = [high - low for high, low in zip(data['High'], data['Low'])]

largest_change_same_day = max(change_same_day)
print(largest_change_same_day)

2.8100000000000023


In [142]:
#5. What was the largest change between any two days (based on Closing Price)?
change_two_days = [abs(x - data['Close'][i - 1]) for i, x in enumerate(data['Close'][1:])]

largest_change_two_days = max(change_two_days)
print(largest_change_two_days)

15.800000000000004


In [146]:
#6. What was the average daily trading volume during this year?
avg_daily_volumn = sum(data['Traded Volume']) / len(data['Traded Volume'])

print(avg_daily_volumn)

89124.33725490196


In [155]:
#7. What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.
def median(lst):
    sortedLst = sorted(lst)
    lstLen = len(lst)
    index = (lstLen - 1) // 2

    if lstLen == 0:
        return print("No element in the list")
    elif (lstLen % 2):
        return sortedLst[index]
    else:
        return (sortedLst[index] + sortedLst[index + 1])/2.0

volume_median = median(data['Traded Volume'])

print(volume_median)

76286.0
